The decorator is a callable that takes another function as argument. The decorator helps to transform the function as follows.

In [ ]:
@decorator
def target():
    print('running target()')

# has the same effect as

def target():
    print('running target')
    
target = decorator(target)

The first crucial fact about decorators is that they have the power to replace the decorated function with a different one. Second crucial fact is that they are exectuted immediately when a module is loaded.

In [2]:
registry = []

def register(func):
    print('running register (%s)'%func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('register -> ', registry)
    f1()
    f2()
    f3()


running register (<function f1 at 0x7f9564048050>)
running register (<function f2 at 0x7f95640b85f0>)


In [3]:

main()

running main()
register ->  [<function f1 at 0x7f9564048050>, <function f2 at 0x7f95640b85f0>]
running f1()
running f2()
running f3()


This emphasis the point that function decorators are executed as soon as the module is imported but the imported functions run only when they are explicitly called. This is what you call `importtime` and `runtime`

## Decorator-Enhanced Strategy Pattern

Decorators offer a way of implementing the `bestprom` functionality from the prev chapter. Here we can use decorators to register each promo code and that way minmise code reuse.

In [5]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

Now in the above examples we saw cases the decorators send back the same function, but most of the decorators do change the function. This is done by defining a inner function and returning that. To understand that better lets look at Closures and varialbe scope rules.

## Variable Scope Rules

In [1]:
def f1(a):
    print(a)
    print(b)
    
f1(4)

4


NameError: name 'b' is not defined

In [2]:
# expect!
# but if we define a global variable b 

b = 10
f1(4)

4
10


In [3]:
def f2(a):
    print(a)
    print(b)
    b = 5
    
f2(10)

10


UnboundLocalError: local variable 'b' referenced before assignment

Why this happens?

when Python compiles the body of the function, it decides that b is a local
variable because it is assigned within the function. The generated bytecode reflects this
decision and will try to fetch b from the local environment. Later, when the call f2(3)
is made, the body of f2 fetches and prints the value of the local variable a , but when
trying to fetch the value of local variable b it discovers that b is unbound.

In [4]:
def f3(a):
    # use the global b
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)

3
10


In [5]:
b

9

In [8]:
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [9]:
dis(f2)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  4          16 LOAD_CONST               1 (5)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## Closues

closue is a function with an extended scope that encompasses nonglobal variables referenced in the body fo the function but not defined there. 

Now to show this in practice lets build a function `avg()` that calculates the average of all the numbers that are called using it.

In [14]:
# a higher order function for this functionality
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [15]:
avg = make_averager()
avg(10)

10.0

In [16]:
avg(11)

10.5

In [17]:
avg(12)

11.0

look at `series` variable. Its accessed in `averager()` function where its is no longer a local variable, but `averager` is still able to call `series.append()`. 

In this case `series` is what we technically call a free variable. It is a variable that is not bound in the local scope.

Lets inspect the function created by make_averager

In [20]:
avg.__code__.co_varnames

('new_value', 'total')

In [21]:
avg.__code__.co_freevars

('series',)

The binding for series is kept in the `__closure__` attribute of the return function avg. Each item in `avg.__closure__` corresponds to a name in `avg.__code__.co_freevars`. These items are `cells`, and they have an attribute called `cell_contents` where the actual values can be found.

In [22]:
avg.__code__.co_freevars

('series',)

In [23]:
avg.__closure__

(<cell at 0x7f3a548da890: list object at 0x7f3a548bfe10>,)

In [24]:
avg.__closure__[0].cell_contents

[10, 11, 12]

To summarize: a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

*Note* that the only situation in which a function may need to deal with external variables
that are nonglobal is when it is nested in another function.